# Chain-of-Thought Prompts

## Preparation

### Set up the OpenAI API

In [50]:
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import util

load_dotenv()

# Load the OpenAI key from .env
client = OpenAI(
    api_key=os.getenv("OPEN_AI_KEY"),
)

### Helper Functions

In [ ]:
def generate_few_shot_responses(few_shot_examples, prompt_template, queries, output_path):
    responses, emotions_list, queries_list = [], [], []

    system_prompt = "You are Paul, a helpful assistant who is very good in expressing emotions."

    emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

    for query in queries:
        for emotion in emotions:
            prompt = prompt_template.format(emotion=emotion, query=query)

            messages = [{"role": "system", "content": system_prompt}]
            messages.extend(few_shot_examples)
            messages.append({"role": "user", "content": prompt})

            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages,
                seed=16,
                temperature=0.0
            )

            response = completion.choices[0].message.content.strip()
            responses.append(response)
            emotions_list.append(emotion)
            queries_list.append(query)

    util.save_dataframe_files(responses, emotions_list, queries_list, output_path)

### Load Queries

In [52]:
all_factual_queries = util.load_file('Queries/factual-queries.txt')
factual_queries = all_factual_queries[10:50]

all_subjective_queries = util.load_file('Queries/subjective-queries.txt')
subjective_queries = all_subjective_queries[10:50]

print(F"{len(factual_queries)} factual queries have been loaded.")
print(F"{len(subjective_queries)} subjective queries have been loaded.")

40 factual queries have been loaded.
40 subjective queries have been loaded.


### Setup Emotions

In [53]:
emotions = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]

### Setup Prompts

In [59]:
factual_prompt = "Answer the following question expressing {emotion}. {query} Reason step-by-step how your response expresses {emotion}. Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."
subjective_prompt = "Write a text of 100 words based on the following task. Your output text should express {emotion}. {query} Reason step-by-step how your response expresses {emotion}. Format your response as a JSON object with the answer to the question that expresses {emotion} as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."

### Load examples

The CoT prompts are based on the most effective Few-Shot prompt *Human*.

In [54]:
df_factual_examples = pd.read_pickle("cot_examples/factual_examples")
df_subjective_examples = pd.read_pickle("cot_examples/subjective_examples")

## Response Generation

### Manual

In [ ]:
########################################## FACTUAL

factual_reasoning = pd.read_pickle("cot_examples/manual_factual_reasoning")
df_factual_examples['reasoning'] = factual_reasoning['reasoning']

factual_examples = []

for i in range(len(df_factual_examples)):
  factual_examples.append({"role": "user", "content": F"Answer the following question expressing {df_factual_examples.loc[i, 'label']}. {df_factual_examples.loc[i, 'query']} Reason step-by-step how your response expresses anger. Format your response as a JSON object with the answer to the question that expresses anger as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."})
  factual_examples.append({"role": "assistant", "content": f"""
    {{
      "response": "{df_factual_examples.loc[i, 'text']}",
      "reasoning": "{df_factual_examples.loc[i, 'reasoning']}"
    }}
    """
  })

output_path = F"few_shot_factual_cot_manual"

generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, output_path)

########################################## SUBJECTIVE

subjective_reasoning = pd.read_pickle("cot_examples/manual_subjective_reasoning")
df_subjective_examples['reasoning'] = subjective_reasoning['reasoning']

subjective_examples = []

for i in range(len(df_subjective_examples)):
  subjective_examples.append({"role": "user", "content": F"Answer the following question expressing {df_subjective_examples.loc[i, 'label']}. {df_subjective_examples.loc[i, 'query']} Reason step-by-step how your response expresses anger. Format your response as a JSON object with the answer to the question that expresses anger as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."})
  subjective_examples.append({"role": "assistant", "content": f"""
    {{
      "response": "{df_subjective_examples.loc[i, 'text']}",
      "reasoning": "{df_subjective_examples.loc[i, 'reasoning']}"
    }}
    """
  })

output_path = F"few_shot_subjective_cot_manual"

generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, output_path)

### Automatic

#### Prompt for Generating Reasoning Texts

We provide the original prompt used for generating the reasoning texts with GPT-3.5-turbo. However, we also added the reasoning texts used in our experiments in the folder cot_examples (auto_factual_reasoning and auto_subjective_reasoning).

In [ ]:
output_path = "cot_exmamples/reasoning_texts"

df_examples = pd.concat([df_factual_examples, df_subjective_examples], axis=0)

responses = []

for _, row in df_examples:
  prompt = F"Explain how the emotion {row['label']} is expressed in the following text. {row['text']}"

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "user", "content": prompt}
    ],
    seed = 16,
    temperature = 0.0
  )

  responses.append(completion.choices[0].message.content)

pd.to_pickle(output_path)

12 list entries are saved to /Users/kerstin/Library/CloudStorage/OneDrive-officehn/MA Wirtschaftsinformatik/Master Thesis/Experiments/Prompt_responses/cot_text_evaluation-without-emotions.txt successfully.


#### Prompting

In [ ]:
########################################## FACTUAL

factual_reasoning = pd.read_pickle("cot_examples/automatic_factual_reasoning")
df_factual_examples['reasoning'] = factual_reasoning['reasoning']

factual_examples = []

for i in range(len(df_factual_examples)):
  factual_examples.append({"role": "user", "content": F"Answer the following question expressing {df_factual_examples.loc[i, 'label']}. {df_factual_examples.loc[i, 'query']} Reason step-by-step how your response expresses anger. Format your response as a JSON object with the answer to the question that expresses anger as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."})
  factual_examples.append({"role": "assistant", "content": f"""
    {{
      "response": "{df_factual_examples.loc[i, 'text']}",
      "reasoning": "{df_factual_examples.loc[i, 'reasoning']}"
    }}
    """
  })

output_path = F"few_shot_factual_cot_automatic"

generate_few_shot_responses(factual_examples, factual_prompt, factual_queries, output_path)

########################################## SUBJECTIVE

subjective_reasoning = pd.read_pickle("cot_examples/automatic_subjective_reasoning")
df_subjective_examples['reasoning'] = subjective_reasoning['reasoning']

subjective_examples = []

for i in range(len(df_subjective_examples)):
  subjective_examples.append({"role": "user", "content": F"Answer the following question expressing {df_subjective_examples.loc[i, 'label']}. {df_subjective_examples.loc[i, 'query']} Reason step-by-step how your response expresses anger. Format your response as a JSON object with the answer to the question that expresses anger as the value to the key 'response'. The reasoning is another value with the key 'reasoning'."})
  subjective_examples.append({"role": "assistant", "content": f"""
    {{
      "response": "{df_subjective_examples.loc[i, 'text']}",
      "reasoning": "{df_subjective_examples.loc[i, 'reasoning']}"
    }}
    """
  })

output_path = F"few_shot_subjective_cot_automatic"

generate_few_shot_responses(subjective_examples, subjective_prompt, subjective_queries, output_path)